# Parte 1: Web Scraping con BeautifulSoup

In [137]:
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd

import re

In [45]:
# Importar librerías para ocultar las keys
# -----------------------------------------------------------------------
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
url = 'https://es.wikipedia.org/wiki/Sendero_de_Gran_Recorrido'

In [5]:
def llamar_api(url):
    """
    Realiza una llamada a una API utilizando la URL proporcionada.

    Parameters:
    -----------
    url (str): La URL de la API que se va a llamar.

    Returns:
    --------
    dict or None: Un diccionario con los datos de respuesta de la API si la llamada fue exitosa (código de estado 200).
                  None si la llamada falló o no se pudo autenticar correctamente.
    """
    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.


In [6]:
response = llamar_api(url)

La llamada a la API nos ha dado una respuesta de tipo: 200


In [8]:
sopa_tabla = BeautifulSoup(response.content, 'html.parser')

In [10]:
tablas = sopa_tabla.find_all("table")

In [16]:
len(tablas) 

2

In [19]:
mi_tabla = tablas[0]

In [21]:
lista_encabezados = mi_tabla.find_all("th")
lista_encabezados


[<th>Identificador</th>,
 <th>Denominación</th>,
 <th>Itinerario</th>,
 <th>Variantes
 </th>]

In [34]:
encabezados_senderos = [columna.text for columna in lista_encabezados]
encabezados_senderos = encabezados_senderos[:3]
encabezados_senderos

['Identificador', 'Denominación', 'Itinerario']

In [26]:
filas_senderos = mi_tabla.find_all("tr")
len(filas_senderos)

181

In [29]:
filas_senderos = filas_senderos[1:]

In [30]:
filas_senderos[0]

<tr>
<td><b><a href="/wiki/GR-1" title="GR-1">GR-1</a></b></td>
<td><a href="/wiki/GR-1" title="GR-1">Sendero Histórico</a></td>
<td><a href="/wiki/Ampurd%C3%A1n" title="Ampurdán">Ampurdán</a> - <a href="/wiki/Ba%C3%B1olas" title="Bañolas">Bañolas</a> - <a href="/wiki/Ripoll" title="Ripoll">Ripoll</a> - <a href="/wiki/Berga" title="Berga">Berga</a> - <a href="/wiki/Graus" title="Graus">Graus</a> - <a href="/wiki/Tierrantona" title="Tierrantona">Tierrantona</a> - <a href="/wiki/Riglos" title="Riglos">Riglos</a> - <a href="/wiki/Biel_(Zaragoza)" title="Biel (Zaragoza)">Biel</a> - <a href="/wiki/Sos_del_Rey_Cat%C3%B3lico" title="Sos del Rey Católico">Sos del Rey Católico</a> - <a href="/wiki/Olite" title="Olite">Olite</a> - <a href="/wiki/Allo_(Navarra)" title="Allo (Navarra)">Allo</a> - <a href="/wiki/Santa_Cruz_de_Campezo" title="Santa Cruz de Campezo">Santa Cruz de Campezo</a> - <a href="/wiki/Bernedo" title="Bernedo">Bernedo</a> - <a href="/wiki/Pe%C3%B1acerrada" title="Peñacerrada">P

In [41]:
resultados_filas = []

for fila in filas_senderos:
    fila_texto = fila.text
    elementos_fila = fila_texto.split("\n")[1:4]
    resultados_filas.append(elementos_fila)

print(resultados_filas[:3])

[['GR-1', 'Sendero Histórico', 'Ampurdán - Bañolas - Ripoll - Berga - Graus - Tierrantona - Riglos - Biel - Sos del Rey Católico - Olite - Allo - Santa Cruz de Campezo - Bernedo - Peñacerrada - San Pantaleón de Losa - Reinosa - Cervera de Pisuerga - Maraña -*- Fonteo -*- Sarria -*- Finisterre'], ['GR-2', 'Sendero La Junquera-San Adrián del Besós', 'La Junquera - Rupit - San Adrián del Besós'], ['GR-3', 'Sendero Central de Cataluña', 'Lérida -*- Balaguer -*- Tremp -*- Puebla de Segur -*- El Pont de Suert -*- Seo de Urgel -*- San Juan de las Abadesas - Manresa - Solsona - Tárrega - Vallbona de las Monjas -*- Borjas Blancas -*- Lérida']] 



In [43]:
df_senderos = pd.DataFrame(resultados_filas)
df_senderos.columns = encabezados_senderos
df_senderos.head()

,Identificador,Denominación,Itinerario
0,GR-1,Sendero Histórico,Ampurdán - Bañolas - Ripoll - Berga - Graus - ...
1,GR-2,Sendero La Junquera-San Adrián del Besós,La Junquera - Rupit - San Adrián del Besós
2,GR-3,Sendero Central de Cataluña,Lérida -*- Balaguer -*- Tremp -*- Puebla de Se...
3,GR-4,Sendero Puigcerdá-Mequinenza,Puigcerdá - Montserrat -*- Mequinenza
4,GR-5,Sendero de los Miradores o de los parques natu...,Sitges - Montserrat - Canet de Mar


In [44]:
df_senderos.tail()

,Identificador,Denominación,Itinerario
175,GR-292,Canales romanos de las Médulas (El Bierzo),Área recreativa Campo de Braña (Las Médulas) -...
176,GR-300,Círculo al Embalse de El Atazar,El Berrueco - Patones de Arriba - El Atazar - ...
177,GR-303,Sierra del Rincón,
178,GR-330,Sendero Costa Blanca Interior,Parque natural del Macizo del Montgó - Parque ...
179,GR-1006,Ruta de los Monteros del Rey (Las Merindades),Espinosa de los Monteros - Medina de Pomar - N...


# Parte 2: Obtención de Datos Climatológicos con la API de AEMET

In [46]:
api_key = os.getenv('api_key_aemet')

In [98]:
url_predicion_especifica = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/montaña/pasada/area/gre1/?api_key={api_key}"

json_prediccion = llamar_api(url_predicion_especifica)

La llamada a la API nos ha dado una respuesta de tipo: 200


In [100]:
endpoint = json_prediccion.json()

In [101]:
endpoint['datos']

'https://opendata.aemet.es/opendata/sh/b1dd16e6'

In [102]:
response = llamar_api(endpoint['datos'])

La llamada a la API nos ha dado una respuesta de tipo: 200


In [103]:
response.json()

[{'origen': {'productor': 'Agencia Estatal de Meteorología - AEMET - Gobierno de España',
   'web': 'http://www.aemet.es',
   'tipo': 'Predicción de montaña',
   'language': 'es',
   'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
   'notaLegal': 'http://www.aemet.es/es/nota_legal'},
  'seccion': [{'apartado': [],
    'lugar': [],
    'parrafo': [{'texto': '(En las 24 horas previas a las 10:00 hora oficial del 2 de julio de 2024)',
      'numero': '1'},
     {'texto': '', 'numero': '2'},
     {'texto': 'TEMPERATURAS MÍNIMAS:', 'numero': '3'},
     {'texto': '5ºC en el puerto de El Pico, 8ºC en El Barco de Ávila, 10ºC en La Covatilla, 11ºC en Muñotello, 14ºC en El Piornal y Hervás 16ºC en Tornavacas y Garganta La Olla.',
      'numero': '4'},
     {'texto': 'TEMPERATURAS MÁXIMAS:', 'numero': '5'},
     {'texto': '29ºC en Garganta La Olla; Tornavacas, El Piornal y El Barco de Ávila; 23ºC en Hervás; 24ºC en Muñotello;

In [106]:
response.json()[0]

{'origen': {'productor': 'Agencia Estatal de Meteorología - AEMET - Gobierno de España',
  'web': 'http://www.aemet.es',
  'tipo': 'Predicción de montaña',
  'language': 'es',
  'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
  'notaLegal': 'http://www.aemet.es/es/nota_legal'},
 'seccion': [{'apartado': [],
   'lugar': [],
   'parrafo': [{'texto': '(En las 24 horas previas a las 10:00 hora oficial del 2 de julio de 2024)',
     'numero': '1'},
    {'texto': '', 'numero': '2'},
    {'texto': 'TEMPERATURAS MÍNIMAS:', 'numero': '3'},
    {'texto': '5ºC en el puerto de El Pico, 8ºC en El Barco de Ávila, 10ºC en La Covatilla, 11ºC en Muñotello, 14ºC en El Piornal y Hervás 16ºC en Tornavacas y Garganta La Olla.',
     'numero': '4'},
    {'texto': 'TEMPERATURAS MÁXIMAS:', 'numero': '5'},
    {'texto': '29ºC en Garganta La Olla; Tornavacas, El Piornal y El Barco de Ávila; 23ºC en Hervás; 24ºC en Muñotello; 23ºC en el puer

In [109]:
response.json()[0].keys()

dict_keys(['origen', 'seccion', 'id', 'nombre'])

In [110]:
response.json()[0]['seccion']

[{'apartado': [],
  'lugar': [],
  'parrafo': [{'texto': '(En las 24 horas previas a las 10:00 hora oficial del 2 de julio de 2024)',
    'numero': '1'},
   {'texto': '', 'numero': '2'},
   {'texto': 'TEMPERATURAS MÍNIMAS:', 'numero': '3'},
   {'texto': '5ºC en el puerto de El Pico, 8ºC en El Barco de Ávila, 10ºC en La Covatilla, 11ºC en Muñotello, 14ºC en El Piornal y Hervás 16ºC en Tornavacas y Garganta La Olla.',
    'numero': '4'},
   {'texto': 'TEMPERATURAS MÁXIMAS:', 'numero': '5'},
   {'texto': '29ºC en Garganta La Olla; Tornavacas, El Piornal y El Barco de Ávila; 23ºC en Hervás; 24ºC en Muñotello; 23ºC en el puerto de El Pico y 16ºC en La Covatilla.',
    'numero': '6'},
   {'texto': 'VIENTO: flojo con intervalos de intensidad moderada.',
    'numero': '7'}],
  'nombre': 'tiempo_pasado'}]

In [132]:
fecha_recop_datos = response.json()[0]['seccion'][0]['parrafo'][0]['texto']
fecha_recop_datos

'(En las 24 horas previas a las 10:00 hora oficial del 2 de julio de 2024)'

In [131]:
temperatura_minima = response.json()[0]['seccion'][0]['parrafo'][2]['numero']
temperatura_minima

'3'

In [130]:
temperatura_maxima = response.json()[0]['seccion'][0]['parrafo'][4]['numero']
temperatura_maxima

'5'

In [129]:
viento = response.json()[0]['seccion'][0]['parrafo'][6]['texto']
viento

'VIENTO: flojo con intervalos de intensidad moderada.'

In [159]:
columnas = [['temp_maxima', 'temp_minima', 'sierra', 'fecha']]

In [150]:
fecha = fecha_recop_datos.replace("(", "").replace(")", "")
anio = fecha[-4:]

patron_mes = r"(?i)\b(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\b"
mes = re.search(patron_mes, fecha_recop_datos)
mes = mes.group()

patron_dia = r"\b\d{1,2}(?=\s+de\s+(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre))"
dia = re.search(patron_dia, fecha_recop_datos)
dia = dia.group()
if len(dia) == 1:
    dia = '0' + dia

fecha = anio + '-' + mes[:3].capitalize() + '-' + dia
fecha


'2024-Jul-02'

In [157]:
fila = [[temperatura_maxima, temperatura_minima, 'sierra de gredos', fecha]]

In [160]:
df_sierra = pd.DataFrame(fila)
df_sierra.columns = columnas
df_sierra.head()

,temp_maxima,temp_minima,sierra,fecha
0,5,3,sierra de gredos,2024-Jul-02


# Creación de la BBDD 

```sql
-- -----------------------------------------------------
-- Esquema rutas
-- -----------------------------------------------------
CREATE SCHEMA IF NOT EXISTS `rutas` DEFAULT CHARACTER SET utf8 ;
USE `rutas` ;

-- -----------------------------------------------------
-- Tabla `rutas`.`rutas`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `rutas`.`rutas` (
  `id_rutas` INT NOT NULL AUTO_INCREMENT,
  `nombre` VARCHAR(100) NOT NULL,
  `donde` VARCHAR(100) NOT NULL,
  `tipo` VARCHAR(100) NOT NULL,
  `duracion` VARCHAR(100) NOT NULL,
  `km` VARCHAR(100) NOT NULL,
  `dificultad` VARCHAR(100) NOT NULL,
  `esfuerzo` VARCHAR(100) NOT NULL,
  `descripcion` VARCHAR(100),
  PRIMARY KEY (`id_rutas`))
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Tabla `rutas`.`clima`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `rutas`.`clima` (
  `id_clima` INT NOT NULL AUTOINCREMENT,
  `temp_maxima` VARCHAR(100) NOT NULL,
  `temp_minima` VARCHAR(100) NOT NULL,
  `sierra` VARCHAR(100) NOT NULL,
  `fecha` DATETIME NOT NULL,
  PRIMARY KEY (`id_clima`))
ENGINE = InnoDB;
```

In [ ]:
# Modelo de código SQL para crear las tablas usando Python

import sqlite3 # Importar módulo para trabajar con SQL desde python

# Crear una conexión a una base de datos en memoria (o en un archivo)
conn = sqlite3.connect(':memory:')  # Usa 'tu_base_de_datos.db' para almacenar en un archivo

# Crear un cursor para ejecutar comandos SQL
cursor = conn.cursor()

# Crear la primera tabla para almacenar datos de rutas
cursor.execute('''
CREATE TABLE rutas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    donde TEXT NOT NULL,
    tipo TEXT NOT NULL,
    duracion TEXT NOT NULL,
    km REAL NOT NULL,
    dificultad TEXT NOT NULL,
    esfuerzo TEXT NOT NULL,
    descripcion TEXT
)
''')

# Crear la segunda tabla para almacenar datos climatológicos
cursor.execute('''
CREATE TABLE clima (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    temp_maxima REAL NOT NULL,
    temp_minima REAL NOT NULL,
    sierra TEXT NOT NULL,
    fecha TEXT NOT NULL
)
''')

# Confirmar los cambios
conn.commit()

# Mostrar las tablas creadas con sus datos
print("Datos en la tabla 'rutas':")
for row in cursor.execute('SELECT * FROM rutas'):
    print(row)

print("\nDatos en la tabla 'clima':")
for row in cursor.execute('SELECT * FROM clima'):
    print(row)

# Cerrar la conexión
conn.close()
